In [1]:
import pandas as pd

In [2]:
interactions = pd.read_csv("../Preprocessing/processed_dataframes/interactions.csv")
recipes = pd.read_csv("../Preprocessing/processed_dataframes/recipes_with_text.csv")
recipes.sort_values(by=["recipe_id"], inplace=True)

Average ratings per recipe

## Train on extended data

In [3]:
import sys
sys.path.append("../")

from torch.utils.data import DataLoader

n_users = 226570+1
n_items = 231637

In [5]:
from SVD.train_funcs.svd import fit_svd, predict_batch_svd
import numpy as np

def rmse(uir_val, predict_func, **kwargs):
    val_preds = predict_func(uir_val[:, :2], **kwargs)
    val_expected = uir_val[:, 2]

    return np.sqrt(1/len(uir_val) * np.sum((val_preds - val_expected)**2))

train_data = pd.read_csv("extended_train.csv")
validation_data = pd.read_csv("../Preprocessing/processed_dataframes/val.csv")
uir_train = train_data.values
uir_val = validation_data.values

In [6]:
fitted_params_svd = fit_svd(
    uir_train, uir_val, n_users, n_items, k=4, 
    α1=0.005, α2=0.005, α3=0.005, α4=0.005, λ1=0.01, λ2=0.1,
    n_iters=75,
)

print(rmse(uir_val, predict_batch_svd, params=fitted_params_svd))

  0%|          | 0/75 [00:00<?, ?it/s]

0.8986538981046014


Get items ids that are not rated by the user

In [7]:
my_user_id = interactions["user_id"].max() + 1
my_ratings = pd.read_csv("my_ratings.csv")

In [8]:
unrated_ids = []
for i in range(n_items):
    if i not in my_ratings["recipe_id"].tolist():
        unrated_ids.append(i)

Predict

In [10]:
unrated_ui_mat = np.array([
    [my_user_id]*len(unrated_ids),
    unrated_ids,
]).T

predictions_for_me = predict_batch_svd(unrated_ui_mat, fitted_params_svd)

Top 10 recommended recipes

In [12]:
id_rating_pair_list = [(prediction, recipe_id) for recipe_id, prediction in zip(unrated_ids, predictions_for_me)]
id_rating_pair_list.sort(reverse=True)

In [13]:
for _, recipe_id in id_rating_pair_list[:10]:
    recipe_data = recipes.iloc[recipe_id]
#     print(recipe_data["name"])
    print("Name:", recipe_data["name"])
    print()
    print(", ".join(eval(recipe_data["steps"])))
    print("-"*50)

Name: blueberry sour cream pie

mix sugar and flour together in bowl, beat in sour cream , egg , vanilla and salt together until smooth, fold in blueberries, pour into pie crust , and bake at 400f for 25 minutes, meanwhile , combine topping ingredients, remove pie from oven after the 25 minutes , and sprinkle on topping, return pie to oven and bake 10 minutes more, chill before serving
--------------------------------------------------
Name: caprese salad tomatoes  italian marinated tomatoes

mix all ingredients together and set on counter stirring occasionally, the salt will bring out the juice of the tomatoes, taste and add more seasonings to your personal taste, you can use fresh basil or oregano if preferred , just remember to chopped finely and use a little less than dried, uses: serve with almost any meal as a side, on top of a green salad, use as a topping for bruschetta--just chop tomatoes instead of slicing, use for caprese salad--add sliced buffalo mozzarella and you're done,